In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here

#if __name__ == "__main__":
#    tf.app.run()

In [3]:
def cnn_mnist(features,labels, mode):
    input_layer=tf.reshape(features['x'],[-1,28,28,1]) # batch size * width * height * channel
    conv1=tf.layers.conv2d(
        inputs=input_layer,
        filters=32, # number of filters
        kernel_size=[5,5], # kernel width * height
        padding='same', # keep the output tensor same width and height as input
        activation=tf.nn.relu)
        # output has shape batchsize* 28*28* 32
    
    pool1 = tf.layers.max_pooling2d(inputs=conv1, 
        pool_size=[2,2], # size of pool filter 
        strides=2) #  subregions extracted by the filter should be separated by 2 pixels in both the width and height 
        # output batch_size, 14, 14, 32, 2*2 filter reduce size 50%
    
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
        # output batchsize* 14*14* 64
    pool2 = tf.layers.max_pooling2d(inputs=conv2, 
        pool_size=[2,2], # size of pool filter 
        strides=2) #  subregions extracted by the filter should be separated by 2 pixels in both the width and height 
        # output batch_size, 7, 7, 64, 2*2 filter reduce size 50%        
        
    # dense layer output: batchsize * 1024   
    pool2_flatten = tf.reshape(pool2, [-1,7*7*64]) #  shape [batch_size, features]
    dense = tf.layers.dense(inputs=pool2_flatten, units=1024, activation=tf.nn.relu) # 1024 units
    dropout= tf.layers.dropout(inputs=dense, rate=0.4, training=mode==tf.estimator.ModeKeys.TRAIN) # training=True, turn on the dropout
    
    # logits layer
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    # predict class
    predictions= {'classes': tf.argmax(input=logits,axis=1),
                  'probabilities': tf.nn.softmax(logits, name='softmax_tensor')}
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    
    # calculate loss
    onehot_labels= tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    # indices: location where value is 1
    # depth: number of classes
    loss=tf.losses.softmax_cross_entropy(logits=logits, onehot_labels=onehot_labels)
    
    # Configure the Training Op
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op)
    
    # evaluate 
    eval_metrics_ops = {'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])}
    return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metrics_ops =eval_metrics_ops)

    

In [7]:
def main():
    # Load training and eval data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    # Create the Estimator
    mnist_classifier=tf.estimator.Estimator(model_fn=cnn_mnist,model_dir='/temp/mnist_model')
    
    # track progress during training with logging hook
    tensors_to_log={'probabilities': 'softmax_tensor'} #  log the probability values from the softmax layer
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)
    
    # train
    train_input_fn = tf.estimator.inputs.numpy_input_fn(x={'x':train_data}, y=train_labels, batch_size=100,num_epochs=None, shuffle=True)
    mnist_classifier.train(input_fn=train_input_fn,steps=2000, hooks=[logging_hook])  
    
    # Evaluate the Model
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": eval_data},y=eval_labels,num_epochs=1,shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)
    
    
    

In [8]:
main()

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST-data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/temp/mnist_model', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /temp/mnist_model\model.ckpt.
INFO:tensorflow:probabilities = [[ 0.09206025  0.10844808  0.06742358  0.10711758  0.102

TypeError: __new__() got an unexpected keyword argument 'eval_metrics_op'